# Segmentation using Lungmask library
Uses https://github.com/JoHof/lungmask
    
Uses material from https://github.com/SimpleITK/SimpleITK

input: 3D CT scan image of the lung (raw image)
output: 3D segmented lung image

In [1]:
import numpy as np
import pandas as pd
import os
import scipy.ndimage as ndimage
from lungmask import mask
import SimpleITK as sitk
from myshow import myshow, myshow3d


In [2]:
input_path = "./submission_data/nifti_input_data/2.000000-ROUTINE-CHEST-NON-CON-70218.nii"
output_path = "./submission_data/output_masks_updated/2.000000-ROUTINE CHEST NON-CON-70218.nii"

In [3]:
# Registration files
# /manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-SITE2-000245/01-15-2002-NA-NA-80471/2.000000-NA-58989
# /manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-SITE2-000245/02-12-2002-NA-NA-23720/5.000000-NA-71388
# input_path = "./submission_data/registration/nifti_files/patient1/5.000000-NA-71388.nii"
# output_path = "./submission_data/registration/segmented_lungs/patient1/5.000000-NA-71388.nii"
# output_path_mask = "./submission_data/registration/lung_masks/patient1/5.000000-NA-71388.nii"

In [4]:
input_image = sitk.ReadImage(input_path)

segmentation = mask.apply(input_image)

origin = input_image.GetOrigin()
spacing = input_image.GetSpacing()
direction = input_image.GetDirection()

INFO:root:No GPU support available, will use CPU. Note, that this is significantly slower!


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 62.40it/s]


In [5]:
output_image = sitk.GetImageFromArray(segmentation)

In [6]:
output_image = sitk.Cast(sitk.RescaleIntensity(output_image), sitk.sitkInt16)
output_image.SetOrigin(origin)
output_image.SetSpacing(spacing)
output_image.SetDirection(direction)


# sitk.WriteImage(sitk.Cast(output_image, sitk.sitkInt16), "patient_test_1_output.nii")
sitk.WriteImage(output_image, output_path)

In [7]:
# My own implementation
input_image_array = sitk.GetArrayFromImage(input_image)
mask = (segmentation != 0).astype(np.uint8)
mask = sitk.GetImageFromArray(mask)
mask.CopyInformation(input_image)

masking_value=0
outside_value=-1500
mf = sitk.MaskImageFilter()
mf.SetMaskingValue(masking_value)
mf.SetOutsideValue(outside_value)
output_image_from_mask =  mf.Execute(input_image, mask)

In [8]:
myshow(output_image_from_mask, "Segmented Lungs")
sitk.WriteImage(output_image_from_mask, output_path)

interactive(children=(IntSlider(value=100, description='z', max=200), Output()), _dom_classes=('widget-interac…